In [1]:
import re
import os
import time
import glob
import platform

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from urlvalidator import validate_url, validate_email, ValidationError

In [2]:
# print(os.getcwd()+"/Drivers/chromedriver_"+platform.system())
# os.getcwd().replace("/","\\")+"\\Drivers\\chromedriver_"+platform.system()+".exe"

In [3]:
# os.path.isdir("/Applications/Google Chrome Canary.app")

In [4]:
browser = None
bno = 0

def getBrowser():
    global bno
    
    bno += 1
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument("--window-size=1920x1080")
    
    os_name = platform.system()
    if os_name == "Windows":
        chrome_driver = os.getcwd().replace("/","\\")+"\\Drivers\\chromedriver_"+os_name+".exe"
    elif os_name == "Darwin":
        chrome_browser = "Google Chrome"
        chrome_dev_browser = "Google Chrome Canary"
        
        # Browser -> /Applications/Google Chrome.app
        # Binary -> /Applications/Google Chrome.app/Contents/MacOS/Google Chrome
        
        # Dev Browser -> /Applications/Google Chrome Canary.app
        # Dev Binary -> /Applications/Google Chrome Canary.app/Contents/MacOS/Google Chrome Canary
        
        if os.path.isdir("/Applications/"+chrome_browser+".app"):
            chrome_options.binary_location = "/Applications/"+chrome_browser+".app/Contents/MacOS/"+chrome_browser
        elif os.path.isdir("/Applications/"+chrome_dev_browser+".app"):
            chrome_options.binary_location = "/Applications/"+chrome_dev_browser+".app/Contents/MacOS/"+chrome_dev_browser
        chrome_driver = os.getcwd()+"/Drivers/chromedriver_"+os_name
    else:
        chrome_driver = os.getcwd()+"/Drivers/chromedriver_"+os_name

    browser = webdriver.Chrome(chrome_options=chrome_options, executable_path=chrome_driver)
    print("Browser - "+str(bno)+" Created Successfully !!")
    return browser


def startBrowsers():
    global browser1, browser2
    browser1 = getBrowser()
    browser2 = getBrowser()
    print()


def closeBrowser1():
    global browser1, bno
    
    if browser1 != None:
        browser1.quit()
        print("Browser - 1 Closed Successfully !!")
    else:
        print("Browser - 1 Not Found !!")
    if bno > 0:
        bno -= 1


def closeBrowser2():
    global browser2, bno
    
    if browser2 != None:
        browser2.quit()
        print("Browser - 2 Closed Successfully !!")
    else:
        print("Browser - 2 Not Found !!")
    if bno > 0:
        bno -= 1


def stopBrowsers():
    closeBrowser2()
    closeBrowser1()


def getNumList(msg="", sep=True):
    nlist = []
    reg = []
    ano = 0
    if msg=="":
        return [0]
    if sep==False:
        reg = re.findall(r'\d+', msg)
    else:
        reg = re.findall(r'\b\d+\b', msg)
    
    for i in reg:
        nlist.append(int(i))
    
    return nlist


def isUrl(link=""):
    try:
        validate_url(link)
    except ValidationError:
        return False
    
    return True
            

In [5]:
getNumList("")

[0]

In [6]:
def getAns(url=""):
    global browser2
    alist = []
    ano = 0
    
    if url=="" or isUrl(url)==False:
        return alist
    
    browser2.get(url)
    body=browser2.find_element_by_tag_name('body')
    
    try:
        ano = browser2.find_elements_by_class_name('answer_count')[0].text
        ano = int(getNumList(ano)[0])
    except IndexError:
        ano = 0
    
    for i in range(ano):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1.5)
    
    n=1
    answers = browser2.find_elements_by_xpath("//span[@class='ui_qtext_rendered_qtext']")
    for answer in answers:
        alist.append((answer.text).replace("\n\n","\n").replace("\n \n","\n").replace("\n","\n\n"))
        
        n += 1
        if n==ano+1:
            break
    
    return alist

In [7]:
def genFName(fname=""):
    fname = fname.split(".")[0]
    if fname == "":
        return fname
    
    if os.path.exists(fname+".txt"):
        if fname[-2] == "_" :
            fname = fname[:-2]
        elif fname[-3] == "_" :
            fname = fname[:-3]
        total = len(glob.glob1(".",fname+"*"))
        fname = fname+"_"+str(total)+".txt"
    else:
        fname = fname+".txt"
    
    return fname

In [8]:
genFName("quora")

'quora.txt'

In [9]:
def writeToTextFile(fname="", res="", write_count=1):
    if fname=="":
        print("Please give a File Name !!")
        return ""
    elif res=="":
        print("Please give a Text to Write !!")
        return ""
    elif write_count == 0:
        fname = fname.split(".")[0]
        fname = genFName(fname)
    else:
        fname = fname.split(".")[0]
        
    append_write = 'w'      # make a new file if not
    if os.path.exists(fname+".txt"):
        append_write = 'a'  # append if already exists
        fname = fname+".txt"
    else:
        fname = genFName(fname)
    
    with open(fname, append_write) as wf:
        wf.write(res)
    
    return fname

In [10]:
# writeToTextFile("quora", "xyz")

In [11]:
comment_1 = '''
def searchQuora(key="", qno=0):
    global browser1
    res = ""
    
    startBrowsers()
    
    if key=="" or qno==0:
        return key
    qno = int(qno)
    search_link = 'https://www.quora.com//search?q='+key.replace(' ','+')
    browser1.get(search_link)
    
    body=browser1.find_element_by_tag_name('body')
    for i in range(10):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        
    questions=browser1.find_elements_by_class_name('question_link')
    
    i=1
    n=1
    for question in questions:
        q = question.text
        l = question.get_property('href')
        ans = getAns(l)
        if ans==[]:
            ans = [""]
        
        res += "\n\n\nQ"+str(i)+".) "+q+"\n\nLink : "+str(l)
        j=1
        for a in ans:
            if a == "":
                res += "\n\nNO ANSWERS FOUND !!"
            else:
                res += "\n\nAns No.-"+str(j)+"\n=========\n"+a
            j+=1
        res+="\n"
        i += 1
        
        n += 1
        if n==qno+1:
            break
            
    stopBrowsers()
    return res
    # return search_link
'''

In [12]:
# print(searchQuora(key="US Mortgage", qno=10))


In [13]:
file_name = ""
def searchQuoraToText(key="", qno=0, fname=""):
    global browser1, file_name
    res = ""
    # fname = "quora"
    write_count = 0
    
    startBrowsers()
    
    if fname=="" or key=="" or qno==0:
        return key
    qno = int(qno)
    search_link = 'https://www.quora.com//search?q='+key.replace(' ','+')
    browser1.get(search_link)
    
    body=browser1.find_element_by_tag_name('body')
    for i in range(qno):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1.5)
        
    questions=browser1.find_elements_by_class_name('question_link')
    
    n=1
    for question in questions:
        q = question.text
        l = question.get_property('href')
        ans = getAns(l)
        if ans==[]:
            ans = [""]
        
        res = "Q$Q$Q\n"+q
        for a in ans:
            if a == "":
                res += "\nA$A$A"
            else:
                res += "\nA$A$A\n"+a
        res+="\nE$E$E\n\n"
        
        fname = writeToTextFile(fname, res, write_count)
        if write_count == 0:
            print("File Created : %s\n" % fname)
        write_count += 1
        if fname=="":
            print("Write to File Unsuccessfull !!\n")
            return
        else:
            print("Q%s. written to : %s" % (write_count, fname))
        
        n += 1
        if n==qno+1:
            break
    
    print()
    write_count = 0
    file_name = fname
    stopBrowsers()

---
### Search Quora for topic and save to a text file

In [20]:
searchQuoraToText(key="US Mortgage", qno=10, fname="US_Mortgage")

Browser - 1 Created Successfully !!
Browser - 2 Created Successfully !!

File Created : US_Mortgage.txt

Q1. written to : US_Mortgage.txt
Q2. written to : US_Mortgage.txt
Q3. written to : US_Mortgage.txt
Q4. written to : US_Mortgage.txt
Q5. written to : US_Mortgage.txt
Q6. written to : US_Mortgage.txt
Q7. written to : US_Mortgage.txt
Q8. written to : US_Mortgage.txt
Q9. written to : US_Mortgage.txt
Q10. written to : US_Mortgage.txt

Browser - 2 Closed Successfully !!
Browser - 1 Closed Successfully !!


---

### Syntax for saving in the text file :
### - - - - - - - - - - - - - - - - - - - - - - -

**Q\$Q\$Q**

_Question No. - **1**_

**A\$A\$A**

_Answer No. - 1_

**A\$A\$A**

_Answer No. - 2_

**A\$A\$A**

_._

**A\$A\$A**

_._


**A\$A\$A**

_Answer No. - N_

**E\$E\$E**

**.**

**.**

**.**

**.**

**.**

**Q\$Q\$Q**

_Question No. - **M**_

**A\$A\$A**

_._

**A\$A\$A**

_Answer No. - N_

**E\$E\$E**

---

In [15]:
def showData(fname=""):
    data = ""
    if os.path.exists(fname):
        with open(fname, "r") as rf:
            for line in rf:
                data += line
        print(data)
    else:
        print("File Not Found : %s" % fname)

In [16]:
showData(file_name)

Q$Q$Q
Why do many middle and lower middle class people in the US mortgage a house then put it up for rent (as an investment) but most wealthy people do not?
A$A$A
The “anyone can invest in real estate” mantra has been a constant fixture in the US. Regardless of the market going up or down, there is no shortage of real estate gurus telling people how to make a killing on property.

Buy foreclosed homes. Renovate and flip. Buy to let. You can find countless guides, websites, ebooks, etc. on these subjects. But I will let you in a secret.

The reason the authors of these systems are selling ebooks and guides is because they cannot make money investing in property.

Back to your question. Everyone likes the pitch which is something like this:

Hate your job? No money at the end of the month?

You can make a killing flipping property or be sitting on the beach while idiots pay you rent.

The wealthy don't have a job and have money at the end of the month. So the pitch only works on the midd

---

<center><h3>END</h3></center>

In [17]:
def getList(fname=""):
    data = ""
    dlist = []
    if os.path.exists(fname):
        data = open(fname, 'r', encoding="iso-8859-15").read()
        # data = data.split("E$E$E")[0].split("Q$Q$Q")[1].split("A$A$A")
        data = data.split("E$E$E")
        data = data[0:-1]
        
        i=0
        for sub_data in data:
            temp = []
            dlist.append([])
            temp = sub_data.split("Q$Q$Q")[1].split("A$A$A")
            for t in temp:
                t = t.replace("\n\n"," ").replace("\n","").replace(u'\x93',"").replace(u'\x94',"")
                dlist[i].append(t)
                while '' in dlist[i]:
                    dlist[i].remove('')
            i += 1
        
            
        return dlist
    else:
        return "File Not Found : %s" % fname

In [18]:
var = getList("quora.txt")

---

### Format

var = \[ 
        \[Q1, a1, a2, .... , an\],
        \[Q2, a1, a2, .... , an\],
                  .
                  .
                  .
        \[Qm, a1, a2, .... , an\]
      \]
 
 var\[0\] => \[Q1, a1, a2, .... , an\]
 
 ---

In [19]:
var[7]

['How much does a US Mortgage Underwriter earn In Mumbai?']

In [ ]:
searchQuoraToText(key="Mortgage Servicing", qno=10, fname="Mortgage_Servicing")

In [ ]:
searchQuoraToText(key="", qno=10, fname="")

In [ ]:
searchQuoraToText(key="", qno=10, fname="")

In [ ]:
searchQuoraToText(key="", qno=10, fname="")

In [ ]:
searchQuoraToText(key="", qno=10, fname="")